In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_pickle('combinedyears.pkl')
df.head()

,Data,Unidade,Grupo EFR,Grupo Rúbrica,Tipo Rúbrica,Data Nascimento,Sexo,hashed_ID_Único
0,14/09/2017,HCIS,ADSE,SERVIÇOS ESPECIAIS CARDIOLOGIA,EXAMES ESPECIAIS,1980.0,Feminino,91e9d2627cd4cc24958988333c4220a3
1,21/08/2017,CCSJM,PARTICULARES,SERVIÇOS E TÉCNICAS GERAIS,EXAMES ESPECIAIS,1980.0,Feminino,91e9d2627cd4cc24958988333c4220a3
2,21/08/2017,CCSJM,PARTICULARES,SERVIÇOS E TÉCNICAS GERAIS,SERVIÇOS E TÉCNICAS GERAIS,1980.0,Feminino,91e9d2627cd4cc24958988333c4220a3
3,18/12/2017,CCTV,ADSE,SERVIÇOS ESPECIAIS UROLOGIA,EXAMES ESPECIAIS,1980.0,Feminino,91e9d2627cd4cc24958988333c4220a3
4,24/04/2017,CCSJM,PARTICULARES,SERVIÇOS ESPECIAIS CARDIOLOGIA,EXAMES ESPECIAIS,1980.0,Feminino,91e9d2627cd4cc24958988333c4220a3


In [3]:
df.shape

(22079182, 8)

## Data Cleaning

In [4]:
# rename columns
columns = ['Date','Hospital','Payer','Specific_Service','Category_of_Service','Birth_Year','Sex','UniqueID']
df.columns = columns

# drop virtual client
df = df[df.UniqueID != '6bb61e3b7bce0931da574d19d1d82c88'] 

df.head(3)

,Date,Hospital,Payer,Specific_Service,Category_of_Service,Birth_Year,Sex,UniqueID
0,14/09/2017,HCIS,ADSE,SERVIÇOS ESPECIAIS CARDIOLOGIA,EXAMES ESPECIAIS,1980.0,Feminino,91e9d2627cd4cc24958988333c4220a3
1,21/08/2017,CCSJM,PARTICULARES,SERVIÇOS E TÉCNICAS GERAIS,EXAMES ESPECIAIS,1980.0,Feminino,91e9d2627cd4cc24958988333c4220a3
2,21/08/2017,CCSJM,PARTICULARES,SERVIÇOS E TÉCNICAS GERAIS,SERVIÇOS E TÉCNICAS GERAIS,1980.0,Feminino,91e9d2627cd4cc24958988333c4220a3


In [5]:
df[df.UniqueID == '00000f7264c27ba6fea0c837ed6aa0aa']

,Date,Hospital,Payer,Specific_Service,Category_of_Service,Birth_Year,Sex,UniqueID
5387198,11/10/2017,HCP,MÉDIS,URGÊNCIA GERAL,URGÊNCIAS,1969.0,Masculino,00000f7264c27ba6fea0c837ed6aa0aa
5387199,11/10/2017,HCP,MÉDIS,ECOGRAFIA,IMAGIOLOGIA,1969.0,Masculino,00000f7264c27ba6fea0c837ed6aa0aa
5366911,11/02/2016,ICDT,MÉDIS,PATOLOGIA CLINICA,PATOLOGIA CLINICA,1969.0,Masculino,00000f7264c27ba6fea0c837ed6aa0aa
5366912,16/05/2016,ICDT,MÉDIS,PATOLOGIA CLINICA,PATOLOGIA CLINICA,1969.0,Masculino,00000f7264c27ba6fea0c837ed6aa0aa
5366913,18/05/2016,HCP,MÉDIS,DERMATOLOGIA,CONSULTA EXTERNA,1969.0,Masculino,00000f7264c27ba6fea0c837ed6aa0aa


### Basic Processing

In [5]:
import numpy as np

# create an age column
df["Year"] = [x.strip()[-4:] for x in df["Date"]] # strip year from date
df["Year"] = df["Year"].astype(np.int16) # convert to int
df["Age"] = df["Year"] - df['Birth_Year'] # subtract birth year to get age

del df["Birth_Year"]
del df['Year']

df.head(3)

,Date,Hospital,Payer,Specific_Service,Category_of_Service,Sex,UniqueID,Age
0,14/09/2017,HCIS,ADSE,SERVIÇOS ESPECIAIS CARDIOLOGIA,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0
1,21/08/2017,CCSJM,PARTICULARES,SERVIÇOS E TÉCNICAS GERAIS,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0
2,21/08/2017,CCSJM,PARTICULARES,SERVIÇOS E TÉCNICAS GERAIS,SERVIÇOS E TÉCNICAS GERAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0


#### Missing Values and Duplicates

In [6]:
# check for missing values
n_records = len(df)
def missing_values_df(df):
    for column in df:
        print("{} | {} | {}".format(
            column, len(df[df[column].isnull()]) / (1.0*n_records), df[column].dtype
        ))

missing_values_df(df)

Date | 0.0 | object
Hospital | 0.0 | object
Payer | 8.823801725007941e-05 | object
Specific_Service | 0.005359508316750203 | object
Category_of_Service | 0.005359508316750203 | object
Sex | 0.0025555052459947276 | object
UniqueID | 0.0 | object
Age | 0.0 | float64


In [7]:
# fill missing sex with missing
df.Sex.fillna('missing',inplace = True)

# fill '-' with missing
df.replace('-','missing',inplace=True)

# drop outlier ages
df = df[df.Age>=0]
df.loc[df.Age > 110, 'Age'] = np.nan

# drop missing values
df.dropna(inplace = True)

df.head(3)

,Date,Hospital,Payer,Specific_Service,Category_of_Service,Sex,UniqueID,Age
0,14/09/2017,HCIS,ADSE,SERVIÇOS ESPECIAIS CARDIOLOGIA,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0
1,21/08/2017,CCSJM,PARTICULARES,SERVIÇOS E TÉCNICAS GERAIS,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0
2,21/08/2017,CCSJM,PARTICULARES,SERVIÇOS E TÉCNICAS GERAIS,SERVIÇOS E TÉCNICAS GERAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0


In [8]:
# verify no missing values
missing_values_df(df)

Date | 0.0 | object
Hospital | 0.0 | object
Payer | 0.0 | object
Specific_Service | 0.0 | object
Category_of_Service | 0.0 | object
Sex | 0.0 | object
UniqueID | 0.0 | object
Age | 0.0 | float64


In [9]:
df.shape

(21950351, 8)

#### Specific Services

In [13]:
# align female sex with gender specific services
df[(df.Specific_Service == "APARELHO GENITAL FEMININO") | (df.Specific_Service == "BLOCO DE PARTOS") |
   (df.Specific_Service == "ECOGRAFIA") | (df.Specific_Service == "ECOGRAFIA OBSTETRICA") |
   (df.Specific_Service == "GINECOLOGIA-OBSTETRÍCIA") | (df.Specific_Service == "GRAVIDEZ E PARTO") |
   (df.Specific_Service == "IMAGIOLOGIA MAMÁRIA") | (df.Specific_Service == "MAMA") |
   (df.Specific_Service == "MAMOGRAFIA") | (df.Specific_Service == "SERVIÇOS ESPECIAIS GINECOLOGIA") |
   (df.Specific_Service == "SERVIÇOS ESPECIAIS OBSTETRICIA") |
   (df.Specific_Service == "URGÊNCIA GINECOLOGIA-OBSTETRÍCIA") | 
   (df.Specific_Service == "URGÊNCIA OBSTETRICIA")].Sex.replace({'Masculino':'Feminino'}).head(0)

Series([], Name: Sex, dtype: object)

In [14]:
# align male sex with gender specific services
df[(df.Specific_Service == "APARELHO GENITAL MASCULINO")].Sex.replace({'Feminino':'Masculino'}).head(0)

Series([], Name: Sex, dtype: object)

In [15]:
# align age with age specific services
index = df[((df.Specific_Service == "CIRURGIA PEDIÁTRICA") | 
           (df.Specific_Service == "MEDICINA ANTI-ENVELHECIMENTO") | 
           (df.Specific_Service == "PEDIATRIA") | (df.Specific_Service == "PEDOPSIQUIATRIA") | 
   (df.Specific_Service == "URGÊNCIA CIRURGIA PEDIÁTRICA") | (df.Specific_Service == "URGÊNCIA PEDIATRIA") | 
   (df.Specific_Service == "URGÊNCIA PEDOPSIQUIATRIA")) & (df.Age > 18)].index.values.tolist()


In [16]:
# drop ages that don't fit pediatric care
df = df.drop(index)

In [17]:
df.shape

(21921171, 8)

#### Reduce File Size

In [18]:
# convert objects to categories to conserve space
df[["Hospital","Payer","Specific_Service",
    "Category_of_Service","Sex"]] = df[["Hospital","Payer","Specific_Service",
                                        "Category_of_Service","Sex"]].astype('category')

# see datatypes
df.dtypes

Date                     object
Hospital               category
Payer                  category
Specific_Service       category
Category_of_Service    category
Sex                    category
UniqueID                 object
Age                     float64
dtype: object

In [19]:
df.shape

(21921171, 8)

In [20]:
#save a copy of df
df2 = df.copy()

### Categorical Variable Grouping

#### Hospitals

In [21]:
df.Hospital.unique()

[HCIS, CCSJM, CCTV, HCP, HCS, ..., CCAL, ICDT, CCSDR, HCV, CCM]
Length: 16
Categories (16, object): [HCIS, CCSJM, CCTV, HCP, ..., ICDT, CCSDR, HCV, CCM]

In [22]:
# categorical grouping for hospitals based on size

# define lists that group hospitals by size
large_hospital = ['HCD','HCIS','HCP']
medium_hospital = ['CCC','CCTV','HCS','HCV','CUFC']
clinic = ['ICDT','CCA','CCB','CCAL','CCSDR','CCMF','CCS','CCM','CCSJM','CLA']

# create a new column (HType) that describes the size of the hospital 
df.loc[df['Hospital'].isin(large_hospital),'HType'] = 'Large'
df.loc[df['Hospital'].isin(medium_hospital),'HType'] = 'Medium'
df.loc[df['Hospital'].isin(clinic),'HType'] = 'Clinic'

df.head(3)

,Date,Hospital,Payer,Specific_Service,Category_of_Service,Sex,UniqueID,Age,HType
0,14/09/2017,HCIS,ADSE,SERVIÇOS ESPECIAIS CARDIOLOGIA,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,Large
1,21/08/2017,CCSJM,PARTICULARES,SERVIÇOS E TÉCNICAS GERAIS,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,Clinic
2,21/08/2017,CCSJM,PARTICULARES,SERVIÇOS E TÉCNICAS GERAIS,SERVIÇOS E TÉCNICAS GERAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,Clinic


In [23]:
# categorical grouping for hospitals based on region

# define lists that group hospital by region
lisbon = ['HCD', 'HCIS','CCC','CCA','CCB','CCTV','CCAL','CCSDR','CCMF','CCS','CCM']
porto = ['HCP','ICDT','CCSJM']
santarem = ['HCS']
setubal = ['CLA']
coimbra = ['CUFC']
viseu = ['HCV']

# create a new column (HRegion) that describes the region of the hospital 
df.loc[df['Hospital'].isin(lisbon),'HRegion'] = 'Lisbon'
df.loc[df['Hospital'].isin(porto),'HRegion'] = 'Porto'
df.loc[df['Hospital'].isin(santarem),'HRegion'] = 'Santarem'
df.loc[df['Hospital'].isin(setubal),'HRegion'] = 'Sentubal'
df.loc[df['Hospital'].isin(coimbra),'HRegion'] = 'Coimbra'
df.loc[df['Hospital'].isin(viseu),'HRegion'] = 'Viseu'

df.head(7)

,Date,Hospital,Payer,Specific_Service,Category_of_Service,Sex,UniqueID,Age,HType,HRegion
0,14/09/2017,HCIS,ADSE,SERVIÇOS ESPECIAIS CARDIOLOGIA,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,Large,Lisbon
1,21/08/2017,CCSJM,PARTICULARES,SERVIÇOS E TÉCNICAS GERAIS,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,Clinic,Porto
2,21/08/2017,CCSJM,PARTICULARES,SERVIÇOS E TÉCNICAS GERAIS,SERVIÇOS E TÉCNICAS GERAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,Clinic,Porto
3,18/12/2017,CCTV,ADSE,SERVIÇOS ESPECIAIS UROLOGIA,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,Medium,Lisbon
4,24/04/2017,CCSJM,PARTICULARES,SERVIÇOS ESPECIAIS CARDIOLOGIA,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,Clinic,Porto
5,04/05/2017,HCIS,ADSE,ENDOSCOPIA,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,Large,Lisbon
6,29/05/2017,HCP,PARTICULARES,ENDOSCOPIA,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,Large,Porto


#### Payer

In [24]:
# read payer as a new dataframe
payer = pd.read_excel('payer mapping.xlsx')
payer.head(3)

,Grupo EFR (payer),Type of Payer,Type
0,PARTICULARES,Out-of-Pocket,Out-of-Pocket
1,ADSE,State Healthcare Subsystem,State
2,MULTICARE,Private Insurance,Private


In [25]:
# create a dictionary that maps each service to a severity
payer_dict = dict(zip(payer['Grupo EFR (payer)'],payer['Type']))

# add a new columns to the data frame (Insurance_Type)
df['Insurance_Type'] = df['Payer'].map(payer_dict)
df.head()

,Date,Hospital,Payer,Specific_Service,Category_of_Service,Sex,UniqueID,Age,HType,HRegion,Insurance_Type
0,14/09/2017,HCIS,ADSE,SERVIÇOS ESPECIAIS CARDIOLOGIA,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,Large,Lisbon,State
1,21/08/2017,CCSJM,PARTICULARES,SERVIÇOS E TÉCNICAS GERAIS,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,Clinic,Porto,Out-of-Pocket
2,21/08/2017,CCSJM,PARTICULARES,SERVIÇOS E TÉCNICAS GERAIS,SERVIÇOS E TÉCNICAS GERAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,Clinic,Porto,Out-of-Pocket
3,18/12/2017,CCTV,ADSE,SERVIÇOS ESPECIAIS UROLOGIA,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,Medium,Lisbon,State
4,24/04/2017,CCSJM,PARTICULARES,SERVIÇOS ESPECIAIS CARDIOLOGIA,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,Clinic,Porto,Out-of-Pocket


###### Category of Service

In [27]:
# categorical grouping for Category of Service

# define lists of general service
emergency = ['URGÊNCIAS']
treatment = ['HOSPITAIS DE DIA','IMUNOHEMOTERAPIA','INTERNAMENTO','Internamento GDHS','MEDICINA DENTARIA',
             'MEDICINA FISICA E REABILITAÇÃO','MEDICINA NUCLEAR','SERVIÇOS E TÉCNICAS GERAIS']
examination = ['ANATOMIA PATOLÓGICA','CHECK-UP','EXAMES ESPECIAIS','GENETICA','IMAGIOLOGIA',
               'PATOLOGIA CLINICA']
appointment = ['CONSULTA EXTERNA']
surgery = ['BLOCOS','CIRURGIAS','EQUIPA CIRURGICA','PACOTES']
others = ['ARQUIVO','AVENÇAS','CONSUMOS','DIVERSOS','FARMACOS','missing']

# create a new column (General_Service) that describes the type of service
df.loc[df['Category_of_Service'].isin(emergency),'General_Service'] = 'Emergency'
df.loc[df['Category_of_Service'].isin(treatment),'General_Service'] = 'Treatment'
df.loc[df['Category_of_Service'].isin(examination),'General_Service'] = 'Examination'
df.loc[df['Category_of_Service'].isin(appointment),'General_Service'] = 'Appointment'
df.loc[df['Category_of_Service'].isin(surgery),'General_Service'] = 'Surgery'
df.loc[df['Category_of_Service'].isin(others),'General_Service'] = 'Others'

In [28]:
df.head(5)

,Date,Hospital,Payer,Specific_Service,Category_of_Service,Sex,UniqueID,Age,HType,HRegion,Insurance_Type,General_Service
0,14/09/2017,HCIS,ADSE,SERVIÇOS ESPECIAIS CARDIOLOGIA,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,Large,Lisbon,State,Examination
1,21/08/2017,CCSJM,PARTICULARES,SERVIÇOS E TÉCNICAS GERAIS,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,Clinic,Porto,Out-of-Pocket,Examination
2,21/08/2017,CCSJM,PARTICULARES,SERVIÇOS E TÉCNICAS GERAIS,SERVIÇOS E TÉCNICAS GERAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,Clinic,Porto,Out-of-Pocket,Treatment
3,18/12/2017,CCTV,ADSE,SERVIÇOS ESPECIAIS UROLOGIA,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,Medium,Lisbon,State,Examination
4,24/04/2017,CCSJM,PARTICULARES,SERVIÇOS ESPECIAIS CARDIOLOGIA,EXAMES ESPECIAIS,Feminino,91e9d2627cd4cc24958988333c4220a3,37.0,Clinic,Porto,Out-of-Pocket,Examination


#### Clean Up Again

In [29]:
# check for null values
df.isnull().any()

Date                   False
Hospital               False
Payer                  False
Specific_Service       False
Category_of_Service    False
Sex                    False
UniqueID               False
Age                    False
HType                  False
HRegion                False
Insurance_Type         False
General_Service        False
dtype: bool

In [37]:
# convert all objects to categories to save space
df['Sex'] = df['Sex'].astype('category')
df['Hospital'] = df['Hospital'].astype('category')
df['HType'] = df['HType'].astype('category')
df['HRegion'] = df['HRegion'].astype('category')
df['Insurance_Type'] = df['Insurance_Type'].astype('category')
df['General_Service'] = df['General_Service'].astype('category')

In [38]:
df.shape

(21921171, 12)

In [39]:
# Percentage of rows dropped
(22079182 - 21921171)/22079182

0.007156560419674968

In [40]:
# create pickle file
df.to_pickle('CleanedTransactionData.pkl')